<a href="https://colab.research.google.com/github/Samadkhani/ImageRecommendation/blob/main/completeCode_AttributedTraits_FinetuneFMVGG19_ImageNet_dataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import glob
from keras.preprocessing.image import load_img
import warnings
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras import optimizers
import os
from tensorflow.keras.applications import VGG19
# from keras import models
# from keras import layers
from scipy import io
import scipy.io as spio
from tensorflow.keras.callbacks import LearningRateScheduler
%matplotlib inline


Img_size = 160
read_images = []
Lable_images = []
list_OpenUser = []
Lable_list_OpenUser = []

# Program extracting first column
import xlrd
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True

OpennesUser = []

loc = ("/content/gdrive/MyDrive/PsychoFlickr_DataSet/AttributedTraits.xlsx")

wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
sheet.cell_value(0, 0)

for i in range(sheet.nrows):
  OpennesUser.append(sheet.cell_value(i, 0)) # 0:Openness , 1: Conscientiousness, 2: Extraversion, 3: Neuroticism, 4: Agreeableness



t_quartile = np.quantile(OpennesUser, 0.75)
print(t_quartile)
first_quartile = np.quantile(OpennesUser, 0.25)
print(first_quartile)


numOpennesUser = 0

high_OpennesUser = 0
low_OpennesUser = 0
list_OpennesUser = []
Lable_list_OpennesUser = []
for i in range(len(OpennesUser)):
  flag = 0
  if OpennesUser[i] >= t_quartile:
    flag = 1
    high_OpennesUser = high_OpennesUser + 1
    list_OpennesUser.append(i)
    Lable_list_OpennesUser.append(1)
  elif  OpennesUser[i]<=first_quartile:
    flag = 1
    low_OpennesUser = low_OpennesUser + 1
    list_OpennesUser.append(i)
    Lable_list_OpennesUser.append(-1)

  if flag == 1:
    numOpennesUser = numOpennesUser + 1

print(numOpennesUser)
print(low_OpennesUser)
print(high_OpennesUser)


print(len(list_OpennesUser))
print(len(Lable_list_OpennesUser))


base_path = '/content/gdrive/MyDrive/PsychoFlickr_DataSet/All_Users'

lst = os.listdir(base_path)
lst.sort()

numImg = 0
l = -1
for i in lst:
    vid_name = os.path.join(base_path, i)
    flag = 0
    l = l + 1
    if OpennesUser[l] >= t_quartile:
      flag = 1
      list_OpenUser.append(i)
      Lable_list_OpenUser.append(1)
      Lab = 1
    elif  OpennesUser[l]<=first_quartile:
      flag = 1
      list_OpenUser.append(i)
      Lable_list_OpenUser.append(0)
      Lab = 0

    if flag == 1:
      #print(vid_name)
      print("numImg")
      print(vid_name)
      print(numImg)
      numImg = 0
      for j in os.listdir(vid_name):  # read all as' inside A
          img_path = os.path.join(vid_name, j)

          numImg = numImg + 1

          # load the image via load_img function
          img = load_img(img_path)
          resized_image = img.resize((Img_size, Img_size))
          img_numpy_array = img_to_array(resized_image)
          read_images.append(img_numpy_array)
          Lable_images.append(Lab)

    elif flag == 0:
      continue

xData = np.array(read_images)
xData.shape

y_Data = np.array(Lable_images)
y_Data.shape

num_classes = 2
y_Data = keras.utils.to_categorical(y_Data, num_classes)
y_Data.shape

# split into train test sets
from sklearn.model_selection import train_test_split
X_train, X_testVal, y_train, y_testVal = train_test_split(xData, y_Data, test_size=0.2, random_state=40)
print(X_train.shape, X_testVal.shape, y_train.shape, y_testVal.shape)

X_test, X_Val, y_test, y_Val = train_test_split(X_testVal, y_testVal, test_size=0.25, random_state=40)
print(X_test.shape, X_Val.shape, y_test.shape, y_Val.shape)


conv_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(Img_size, Img_size, 3))


model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu', input_dim=5 * 5 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))


conv_base.trainable = False

print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))


conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block4_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False


print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

batch_size = 30
num_epochs = 20

def scheduler(epoch):
    if epoch < 5:
        return 2e-7
    elif epoch < 20:
        return 2e-6


learning_rate_scheduler = LearningRateScheduler(scheduler, verbose=1)


def run_experiment(model):
    optimizer = optimizers.RMSprop()

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.CategoricalCrossentropy(
            from_logits=True, label_smoothing=0.1
        ),
        metrics=[
            keras.metrics.CategoricalAccuracy(name="accuracy"),
            keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=X_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data= (X_Val, y_Val),
        callbacks=[checkpoint_callback, learning_rate_scheduler],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history , model


cct_model = model
history , model= run_experiment(cct_model)


plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")


plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train and Validation Losses Over Epochs", fontsize=14)
plt.legend()
plt.grid()
plt.show()



# UserImages

import os
import numpy as np
import keras
from keras.layers import *
from keras.models import *
from keras.activations import *
import keras.backend as K
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def get_datatest():
    X = []
    y = []
    k = 0

    for j in os.listdir('/content/gdrive/My Drive/UserImages/'):
      for i in os.listdir('/content/gdrive/My Drive/UserImages/'  + str(j) + '/'):
        name = '/content/gdrive/My Drive/UserImages/'  + str(j) + '/'  + str(i)
        print(name)
        try :
          img = Image.open(name)
          k = k + 1
          print(k)
        except Exception:
          continue
        img = img.resize((Img_size, Img_size)).convert("RGB")
        X.append(np.array(img))

    X = np.array(X)
    return X , k

UserImages , k = get_datatest()
print(UserImages.shape)


predict_UserImages = model.predict(UserImages)
print(predict_UserImages.shape)




%cd /content/gdrive/My Drive/Colab Notebooks
io.savemat("predict_Personality_O_FMfinetunvgg19_Data2.mat", {"array": predict_UserImages})
